This script serves as a sanity check on the representation level.
The different data representations are used to classify call types as a proof of sufficient information.

DFA (PAFs), simple MLPs (LFCCs) and CNNs (spectrograms) are used to classify the different call types.

In [1]:
import pandas as pd
import numpy as np
import json
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler

In [2]:
RANDOM_SEED = 42
random.seed(42)
np.random.seed(42)

In [3]:
calls_df = pd.read_csv('features_and_spectrograms.csv')
calls_df['log_padded_spectrogram'] = calls_df['log_padded_spectrogram'].apply(lambda x: np.array(json.loads(x)))

In [14]:
print(len(calls_df[calls_df.isna().any(axis=1)]))
calls_df = calls_df.dropna()
calls_df = calls_df[calls_df['call_type'] != 'unknown']
calls_df.info()

0
<class 'pandas.core.frame.DataFrame'>
Index: 6323 entries, 0 to 7282
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   track_ID                6323 non-null   object 
 1   clip_ID                 6323 non-null   object 
 2   goose_ID                6323 non-null   object 
 3   call_type               6323 non-null   object 
 4   waveform                6323 non-null   object 
 5   sr                      6323 non-null   int64  
 6   filepath                6323 non-null   object 
 7   lfccs                   6323 non-null   object 
 8   peak                    6323 non-null   float64
 9   duration                6323 non-null   float64
 10  log_padded_spectrogram  6323 non-null   object 
 11  log_padded_lfccs        6323 non-null   object 
 12  f0mean                  6323 non-null   float64
 13  f0range                 6323 non-null   float64
 14  f0min                   6323 non-null   flo

In [15]:
le = LabelEncoder()
le.fit(calls_df["call_type"])
calls_df["encoded_call_type"] = le.transform(calls_df["call_type"])

## DFA

In [24]:
features_list = ["duration", "f0mean", "f0range", "f0min", "f0max", "f0std_dev", "mean_slope", "f0_q1", "f0_q2", "f0_q3", "hnr", "centr_s", "skew_s", "kurt_s", "std_s", "centr_t", "skew_t", "kurt_t", "std_t", "avg_f_form1"]

In [26]:
X = calls_df[features_list].values
y = calls_df['call_type'].values

In [27]:
X_scaled = StandardScaler().fit_transform(X)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=RANDOM_SEED)

In [29]:
lda = LDA()
lda.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [30]:
y_pred = lda.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Accuracy:", accuracy_score(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

       alarm       0.05      0.07      0.06        14
     contact       0.59      0.07      0.13       228
   departure       0.84      0.79      0.82       214
    distance       0.97      0.91      0.94        98
 recruitment       0.70      0.95      0.81       651
     triumph       0.31      0.22      0.25        60

    accuracy                           0.72      1265
   macro avg       0.58      0.50      0.50      1265
weighted avg       0.70      0.72      0.66      1265

Accuracy: 0.71699604743083


In [31]:
coef_df = pd.DataFrame(lda.coef_, columns=features_list, index=lda.classes_)

print("Feature coefficients per class:")
print(coef_df)

Feature coefficients per class:
             duration    f0mean   f0range     f0min     f0max  f0std_dev  \
alarm       -1.330481  9.431904  0.787616 -1.115124 -0.463380  -1.005628   
contact     -0.931789  0.580439 -0.983708  0.176062 -0.637279   0.956812   
departure    1.456176 -1.002021  2.125533 -0.822594  0.935061  -2.079052   
distance     3.787620 -5.837739  1.841642  0.812536  2.334597  -2.369426   
recruitment -0.869865  0.667462 -0.548858  0.060262 -0.393518   0.613975   
triumph      1.825007  0.446807 -0.244618  0.239619  0.037258   0.464918   

             mean_slope     f0_q1     f0_q2     f0_q3       hnr   centr_s  \
alarm         -0.456635 -3.783944 -3.216282 -0.620589  0.022479  1.532100   
contact        0.347498  0.032176  0.005285 -0.779599 -0.112713 -0.463999   
departure     -0.685320 -0.047305  0.440093  1.498787  0.285097  1.286958   
distance      -0.860231  1.228300 -1.074718  4.222434  1.012139 -0.148075   
recruitment    0.252937 -0.083731  0.121774 -0.758

In [32]:
abs_coef = coef_df.abs().mean(axis=0)
print("Feature Importance Ranking:")
print(abs_coef.sort_values(ascending=False))

Feature Importance Ranking:
centr_t        4.346632
f0mean         2.994396
duration       1.700156
f0_q3          1.394945
f0std_dev      1.248302
f0range        1.088663
f0_q1          0.869533
f0_q2          0.834594
f0max          0.800182
std_s          0.713223
kurt_t         0.694141
skew_t         0.675752
centr_s        0.657112
skew_s         0.631294
avg_f_form1    0.551747
f0min          0.537699
mean_slope     0.471325
std_t          0.380202
hnr            0.334370
kurt_s         0.181882
dtype: float64


## MLP

## CNN